# QUICKSTART

## Working with data

Dataset: store samples and lables, contains popular datasets \
DataLoader: wraps an iterable around Dataset

In [4]:
import torch
from torch import nn 
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Dataset object includes 2 args: transform and target_transform to modify samples and labels respectively.

In [6]:
# Download training data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# Download test data
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



Pass the dataset as an argument to DataLoader \
This wraps an iterable over the dataset, supporting automatic batching, sampling, shuffling and multiprocess data loading. \
Here, define a batch size of 64 --> Each element in the dataloader iterable will return a batch of 64 features and labels.

In [7]:
batch_size = 64

# create dataloaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


More on loading data: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

## Create Models

- To create a neural network, create a class that inherits from [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). 
- __init__ : define layers
- forward: how data will pass through the network
- you can set device to CPU, GPU or MPS

In [8]:
# get gpu if available
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# define model

class NeuralNetwork(nn.Module):

    # define architecture
    def __init__(self):
        super().__init__()
        self.flatten= nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    # define data passing through network
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


More on building neural networks: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

## Optimizing Model Parameters

To train a model, we need a [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](https://pytorch.org/docs/stable/optim.html).

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

- In a single training loop, model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model's parameters.
- model.train() puts the model into training mode, which is important for defining certain layer's functions, such as dropout and batch normalization layers
- Pytorch adds gradients in optimizer's grad attribute. This is useful in certain situations such as summing gradients across multiple mini-batches. However, in standart training loop, it's desired to use gradients for only the current batch. Thus, reset the grad attribute by: optimizer.zero_grad()


In [10]:
def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    model.train() 

    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device) # initially loaded into cpu's RAM. 

        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)
        
        # backpropagate
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # print the loss in every 100 batch
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        

- Also, check the model's performance against the test dataset -validation set, to be accurate-.
- model.eval(): puts the model into evaluation state
- torch.no_grad(): disable gradient calculation which is not needed --> save memory
- .item():  convert the loss from tensor to Python number

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    bacthes = len(dataloader)
    model.eval() 
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= bacthes
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Train and test the model in a loop

In [13]:
epochs = 10
for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Finished!")

Epoch 1
-------------------------------
loss: 2.292928  [   64/60000]
loss: 2.284130  [ 6464/60000]
loss: 2.267545  [12864/60000]
loss: 2.264941  [19264/60000]
loss: 2.246005  [25664/60000]
loss: 2.207671  [32064/60000]
loss: 2.224888  [38464/60000]
loss: 2.186148  [44864/60000]
loss: 2.185392  [51264/60000]
loss: 2.149645  [57664/60000]
Test Error: 
 Accuracy: 38.2%, Avg loss: 2.150157 

Epoch 2
-------------------------------
loss: 2.157645  [   64/60000]
loss: 2.147261  [ 6464/60000]
loss: 2.089336  [12864/60000]
loss: 2.110262  [19264/60000]
loss: 2.062463  [25664/60000]
loss: 1.991277  [32064/60000]
loss: 2.028081  [38464/60000]
loss: 1.944594  [44864/60000]
loss: 1.956572  [51264/60000]
loss: 1.876418  [57664/60000]
Test Error: 
 Accuracy: 58.9%, Avg loss: 1.884309 

Epoch 3
-------------------------------
loss: 1.916875  [   64/60000]
loss: 1.882778  [ 6464/60000]
loss: 1.767683  [12864/60000]
loss: 1.814804  [19264/60000]
loss: 1.703718  [25664/60000]
loss: 1.651044  [32064/600

More about training the model: https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

## Saving & Loading the Model


### Save
Serialize the internal state dictionary

In [14]:
torch.save(model.state_dict(), "trained_model")

### Load
Recreate the modelstructure and load the state dictionary

In [15]:
new_model = NeuralNetwork().to(device)
new_model.load_state_dict(torch.load("trained_model"))

<All keys matched successfully>

##### Example to test the loaded model

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

new_model.eval()
with torch.no_grad():
    for i in range(10):
        x, y = test_data[i][0], test_data[i][1]
        x = x.to(device)
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Shirt", Actual: "Shirt"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Shirt", Actual: "Coat"
Predicted: "Coat", Actual: "Shirt"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"


More on saving & loading the model: https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html